In [1]:
import numpy as np
import copy
import open3d as o3d
from sklearn.decomposition import PCA
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
def ransac2(pcd, thresh=0.1,kn=2,epoch=1000) :
    """
    points = np.array(N,3)
    """
    t1 = time.time()
    pts = np.array(pcd.points)
    best_n_inliers = 0
    i =0
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    best_plane = [0,0,0,0]
    best_inlier_mask = np.zeros(len(pts))
    while (i < epoch) :
        # TODO Générer un plan à partir de curr_pnt
        # TODO Inlier/outlier 
        t2 = time.time()
        curr_pnt = pts[np.random.randint(len(pts))]
        [k, idx, _] = pcd_tree.search_knn_vector_3d(curr_pnt, kn)
        pca = PCA(3)
        pca.fit(pts[list(idx)])
        mean = pca.mean_
        normal = pca.components_[2,:]
        d = np.dot(normal,mean)
        plane = [normal[0],normal[1],normal[2],d]
        t3 = time.time()
        dist_pt     = abs(np.dot(normal[:3],pts.T) + d / np.linalg.norm(normal))
        inlier_mask = np.less_equal(dist_pt,thresh)
        t4 = time.time()
        
        n_inliers = np.sum(inlier_mask)
        if n_inliers> best_n_inliers:
            best_plane = plane
            best_n_inliers = n_inliers
            best_inlier_mask = inlier_mask
        i += 1
        t5 = time.time()
    return best_plane,list(np.where(best_inlier_mask)[0])

In [31]:
def ransac(pcd, thresh_d=0.1,thresh_n=0.9,kn=2,epoch=1000) :
    """
    points = np.array(N,3)
    """

    t1 = time.time()
    pts = np.array(pcd.points)
    
    best_n_inliers = 0
    i =0
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    
    best_plane = [0,0,0,0]
    best_inlier_mask = np.zeros(len(pts))

    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    pcd_n = np.asarray(pcd.normals)
    while (i < epoch) :
        # TODO Générer un plan à partir de curr_pnt
        # TODO Inlier/outlier 
        t2 = time.time()
        curr_pnt = pts[np.random.randint(len(pts))]
        [k, idx, _] = pcd_tree.search_knn_vector_3d(curr_pnt, kn)
        pca = PCA(3)
        pca.fit(pts[list(idx)])
        mean = pca.mean_
        normal = pca.components_[2,:]
        d = np.dot(normal,mean)
        plane = [normal[0],normal[1],normal[2],d]
        t3 = time.time()
        dist_pt     = abs(np.dot(normal[:3],pts.T) + d / np.linalg.norm(normal))
        if np.linalg.norm(normal) > 1 :
            print(np.where(abs(np.dot(pcd_n,normal))>1))
            1/0
        inlier_mask = np.less_equal(dist_pt,thresh_d)*np.greater_equal(np.abs(np.dot(pcd_n,normal)),thresh_n)
        t4 = time.time()
        
        n_inliers = np.sum(inlier_mask)
        if n_inliers> best_n_inliers:
            best_plane = plane
            best_n_inliers = n_inliers
            best_inlier_mask = inlier_mask
        i += 1
        t5 = time.time()
    return best_plane,list(np.where(best_inlier_mask)[0])

In [32]:
def extract_planes(pcd) :
    pcd_plane = pcd.voxel_down_sample(voxel_size=0.1)
    #pcd_plane = copy.deepcopy(pcd)
    planes = []
    inlierss = []
    while(len(planes)<7 and len(pcd_plane.points) > 100) :
        plane_model, inliers = ransac(pcd_plane,thresh_d = 0.1, kn = 500, epoch=1000)
        # plane_model, inliers = pcd_plane.segment_plane(distance_threshold=0.1,
        #                                  ransac_n=3,
        #                                  num_iterations=10000)
        [a, b, c, d] = plane_model
        print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
        inlier_cloud = pcd_plane.select_by_index(inliers)
        inlier_cloud.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        inlier_cloud.paint_uniform_color([1.0, 0, 0])
        
        outlier_cloud = pcd_plane.select_by_index(inliers, invert=True)
        outlier_cloud.paint_uniform_color([0, 1.0, 0])
        
        o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                        zoom=0.8,
                                        front=[-0.4999, -0.1659, -0.8499],
                                        lookat=[2.1813, 2.0619, 2.0999],
                                        up=[0.1204, -0.9852, 0.1215])

        pcd_plane = outlier_cloud
        planes.append(plane_model)
        inlierss.append(inlier_cloud)

        # TODO remove points according to the plane
    
    o3d.visualization.draw_geometries(inlierss+[outlier_cloud],
                                        zoom=0.8,
                                        front=[-0.4999, -0.1659, -0.8499],
                                        lookat=[2.1813, 2.0619, 2.0999],
                                        up=[0.1204, -0.9852, 0.1215])
    return planes

In [33]:
target_pcd = o3d.io.read_point_cloud("hough-plane-python-master\RES\map_go_5.pcd")
extract_planes(target_pcd)

(array([], dtype=int64),)


ZeroDivisionError: division by zero